In [1]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [2]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') 
        


In [3]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [6]:
def get_quarter(month):
    if month in (1, 2, 3):
        return 1
    elif month in (4, 5, 6):
        return 2
    elif month in (7, 8, 9):
        return 3
    else:
        return 4

melb_df['quarter'] = melb_df['MonthSale'].apply(get_quarter)
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend,quarter
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1,2
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0,2
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0,2
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0,2


In [7]:
melb_df['quarter'].value_counts()

quarter
3    4873
2    4359
4    2329
1    2019
Name: count, dtype: int64

In [10]:
melb_df.sort_values(by='AreaRatio', ascending=[False], ignore_index=True).iloc[1558]

Suburb                       Elsternwick
Rooms                                  2
Type                               house
Price                           716000.0
Method                                 S
SellerG                    hockingstuart
Date                 2017-03-06 00:00:00
Distance                             7.2
Postcode                            3185
Bedroom                                2
Bathroom                               1
Car                                    1
Landsize                             0.0
BuildingArea                       126.0
CouncilArea                    Glen Eira
Lattitude                      -37.89438
Longtitude                     145.00851
Regionname         Southern Metropolitan
Propertycount                       4898
MeanRoomsSquare                     25.2
AreaRatio                            1.0
MonthSale                              3
AgeBuilding                           47
WeekdaySale                            0
StreetType      

In [11]:
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'], ascending=[True, False], ignore_index=True).iloc[18]


Suburb                     Brighton East
Rooms                                  3
Type                           townhouse
Price                          1300000.0
Method                                PI
SellerG                           Buxton
Date                 2016-09-17 00:00:00
Distance                            10.7
Postcode                            3187
Bedroom                                3
Bathroom                               2
Car                                    2
Landsize                           337.0
BuildingArea                       222.0
CouncilArea                      Bayside
Lattitude                       -37.9196
Longtitude                      145.0205
Regionname         Southern Metropolitan
Propertycount                       6938
MeanRoomsSquare                    27.75
AreaRatio                      -0.205725
MonthSale                              9
AgeBuilding                           16
WeekdaySale                            5
StreetType      

In [4]:
melb_df.groupby('Rooms')['Price'].mean()

Rooms
1     4.338245e+05
2     7.750812e+05
3     1.076081e+06
4     1.445282e+06
5     1.870260e+06
6     1.849366e+06
7     1.920700e+06
8     1.602750e+06
10    9.000000e+05
Name: Price, dtype: float64

In [6]:
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

Regionname
Western Victoria              0.011579
Southern Metropolitan         0.043080
Eastern Metropolitan          0.047890
Northern Metropolitan         0.049639
Western Metropolitan          0.051251
South-Eastern Metropolitan    0.073411
Northern Victoria             0.084455
Eastern Victoria              0.147067
Name: Lattitude, dtype: float64

In [11]:
melb_df[(pd.to_datetime('2017-05-01') <= melb_df['Date'])|( melb_df['Date']<= pd.to_datetime('2017-09-01'))].groupby('SellerG')['Price'].sum().sort_values()

SellerG
LITTLE           2.199300e+07
Burnham          2.614650e+07
Moonee           2.878050e+07
Thomson          2.981600e+07
Bells            3.038450e+07
Alexkarbon       3.465559e+07
Buckingham       3.881160e+07
Eview            4.065124e+07
McDonald         4.205350e+07
Purplebricks     4.360900e+07
Philip           4.703580e+07
HAR              4.818300e+07
O'Brien          4.999551e+07
C21              5.350800e+07
Douglas          6.429010e+07
Rendina          6.600628e+07
Cayzer           6.867000e+07
RW               7.013100e+07
Nick             7.531900e+07
Collins          7.674250e+07
Chisholm         7.834425e+07
Love             7.894540e+07
Raine            8.946020e+07
YPA              9.273035e+07
Stockdale        1.064608e+08
Williams         1.156803e+08
Village          1.235205e+08
Harcourts        1.328355e+08
Gary             1.671684e+08
Sweeney          1.805233e+08
Hodges           2.062959e+08
Miles            2.130829e+08
Jas              2.143306e+08
Ka

In [13]:
pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc=['median'],
    fill_value=0
)
pivot

median                 
Type   house townhouse   unit
Rooms                        
1      126.0      88.0   69.5
2      126.0     114.0  110.0
3      126.0     126.0  126.0
4      141.0     159.5  126.0
5      177.0     152.0    0.0
6      126.0       0.0  171.0
7      216.5       0.0    0.0
8      126.0       0.0  126.0
10     126.0       0.0    0.0

In [25]:
pivot2 = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc=['median'],
    fill_value=0
)
pivot2['median']['unit'].sort_values()

SellerG
Burnham          305000
Sweeney          381000
Stockdale        410000
Village          420000
Love             440000
YPA              443500
LITTLE           447000
Raine            447500
Jas              455000
Bells            457500
Rendina          459000
Brad             465000
Douglas          494500
Thomson          495000
Ray              497500
Harcourts        500000
Barry            502000
McDonald         502500
HAR              518000
Nelson           530000
other            535000
McGrath          542500
Greg             547500
Biggin           548000
Gary             550000
Buckingham       553000
Miles            555000
Alexkarbon       560000
hockingstuart    566000
Williams         577000
Purplebricks     582500
Jellis           591750
Collins          592000
Woodards         600000
Eview            602000
Hodges           605500
O'Brien          618500
Moonee           622000
RW               625500
C21              625500
RT               630000
Philip  